某答题闯关节目设置了 m 道题目（编号 1 到 m），节目规则要求将这些题目分为两轮闯关赛，每轮至少包含 1 道题，所有题目必须分配到两轮中。
共有 n 名选手参加比赛，每位选手能答对的题目范围是连续的一段（用 "LR" 描述，表示该选手能答对第 L 剑第 R 号题）。
选手只要能答对某一轮的所有题目（即该轮题目范围完全被选手的答题范围覆盖），就能晋级到复赛阶段。为了节目效果，导演想让尽可能多的选手能够进入复赛。请问在所有可能的题目分配方式中，最多有多少名选手可以晋级？
输入描述：
第一行包含两个正整数 n 和 m（1≤n≤50000，1≤m≤100000）；
接下来 n 行，每行有两个正整数 Li 和 Ri，表示第 i 名选手能答对的题目范围（1≤Li≤Ri≤m）。
输出描述：
输出一个整数，表示最多可以晋级的选手数量。
样例输入：
4 8
4 7
1 4
5 8
2 5
样例输出：
3
提示：
有4个参赛选手，8道试题，
第一位选手可以作答4~7题
第二位选手可以作答1~4题
第三位选手可以作答5~8题
第四位选手可以作答2~5题
此时有多种情况，其中一种可能的情况为：将两轮闯关的题目分为4和除4外的其他题，此时选手1、2、4可在第一轮得到满分，进入复赛。

In [5]:
# TLE 64%
from typing import List

class Solution:
    def maxAdvance(self, n: int, m: int, intervals: List[List[int]]) -> int:
        flags = [0] * m
        ans = 0
        for start, end in intervals:
            for i in range(start, end + 1):
                flags[i - 1] += 1
                if flags[i - 1] > ans:
                    ans = flags[i - 1]
        return ans

In [2]:
from typing import List

class Solution:
    def maxAdvance(self, n: int, m: int, intervals: List[List[int]]) -> int:
        # 1) 计算每个题号被多少区间覆盖（点覆盖数）
        diff = [0] * (m + 3)
        for L, R in intervals:
            diff[L] += 1
            diff[R + 1] -= 1
        cover = [0] * (m + 2)
        cur = 0
        max_point = 0
        for i in range(1, m + 1):
            cur += diff[i]
            cover[i] = cur
            if cur > max_point:
                max_point = cur

        # 2) 处理“连续切分”情况：
        #    prefixR[t] = #区间满足 L == 1 且 R >= t  （能覆盖 [1,t] 的区间数）
        #    prefixL[x] = #区间满足 R == m 且 L <= x   （能覆盖 [x,m] 的区间数）
        cntR_when_L1 = [0] * (m + 2)   # cntR[r] = 个数，满足 (L==1 and R==r)
        cntL_when_Rm = [0] * (m + 2)   # cntL[l] = 个数，满足 (R==m and L==l)
        full_count = 0                 # 满足 L==1 and R==m 的区间数（被两边同时计到，要减去一次）

        for L, R in intervals:
            if L == 1:
                cntR_when_L1[R] += 1
            if R == m:
                cntL_when_Rm[L] += 1
            if L == 1 and R == m:
                full_count += 1

        prefixR = [0] * (m + 3)
        s = 0
        # prefixR[t] = sum_{r >= t} cntR_when_L1[r]
        for t in range(m, 0, -1):
            s += cntR_when_L1[t]
            prefixR[t] = s

        prefixL = [0] * (m + 3)
        s = 0
        # prefixL[x] = sum_{l <= x} cntL_when_Rm[l]
        for x in range(1, m + 1):
            s += cntL_when_Rm[x]
            prefixL[x] = s

        best_cut = 0
        for t in range(1, m):  # 切分点 t，把题分为 [1..t] and [t+1..m]
            # 能晋级的选手数 = #cover [1,t] + #cover [t+1,m] - #cover both([1,m])
            candidates = prefixR[t] + prefixL[t+1] - full_count
            if candidates > best_cut:
                best_cut = candidates

        return max(max_point, best_cut)

好的，这是一个经典的算法问题，可以通过分析题目分区的结构来简化问题，并使用高效的数据结构来求解。

### 问题分析

首先，我们来理解选手晋级的条件。一个选手能晋级，当且仅当他/她能够答对某一轮比赛中的所有题目。设某位选手的答题范围是 `[L, R]`，某轮比赛的题目集合是 `S`。该选手能晋级，则要求 `L <= min(S)` 并且 `R >= max(S)`，其中 `min(S)` 和 `max(S)` 分别是该轮比赛题目编号的最小值和最大值。

我们的目标是找到一个将所有题目 `{1, 2, ..., m}` 划分为两个非空集合 `S1` 和 `S2` 的方案，使得能够晋级的总人数最多。晋级总人数是满足以下条件之一的选手数量：

1.  `L <= min(S1)` 且 `R >= max(S1)`
2.  `L <= min(S2)` 且 `R >= max(S2)`

### 简化划分方案

直接遍历所有可能的集合划分是不现实的。问题的关键在于，选手的晋级只与每轮题目编号的**最小和最大值**有关。我们可以对划分方案的结构进行分析。

不失一般性，我们假设题目 1 被分在第一轮，即 `1 ∈ S1`，因此 `min(S1) = 1`。现在我们对题目 `m` 的位置进行分类讨论：

**情况一：题目 `m` 被分在第二轮 (`m ∈ S2`)**

  * 此时 `min(S1) = 1`, `max(S2) = m`。
  * 设 `max(S1) = k` 且 `min(S2) = j`。由于 `S1` 和 `S2` 是 `{1, ..., m}` 的一个划分，可以证明，为了使两轮题目的边界 `[1, k]` 和 `[j, m]` "最有效"（即覆盖最多选手），最优的划分方式是连续的区间划分。也就是说，`S1 = {1, 2, ..., k}` 且 `S2 = {k+1, ..., m}`。
  * 在这种情况下，选手晋级的条件是：
      * 通过第一轮：`L <= 1` 且 `R >= k`
      * 或通过第二轮：`L <= k+1` 且 `R >= m`
  * 我们可以遍历所有可能的分割点 `k` (从 `1` 到 `m-1`)，计算每种分割下的晋级人数，并取最大值。

**情况二：题目 `m` 也被分在第一轮 (`m ∈ S1`)**

  * 此时 `min(S1) = 1`, `max(S1) = m`。
  * 第二轮 `S2` 则是 `{1, ..., m}` 中除去 `S1` 剩下的非空集合。
  * 选手晋级的条件是：
      * 通过第一轮：`L <= 1` 且 `R >= m` (能答对所有题目)
      * 或通过第二轮：`L <= min(S2)` 且 `R >= max(S2)`
  * 为了最大化晋级人数，我们应该让第二轮的条件尽可能容易满足。当 `min(S2)` 尽可能大，`max(S2)` 尽可能小时，`[min(S2), max(S2)]` 这个区间最短，覆盖它的选手就可能越多。最极端的情况是 `S2` 只包含一个题目 `p`，即 `S2 = {p}`。此时 `min(S2) = max(S2) = p`。
  * 在这种情况下，晋级条件变为：
      * 通过第一轮：`L <= 1` 且 `R >= m`
      * 或通过第二轮：`L <= p` 且 `R >= p` (即答题范围包含题目 `p`)
  * 由于能通过第一轮的选手（答对所有题）也必然能通过第二轮（答对题目`p`），所以总晋级人数就是能答对题目 `p` 的人数。
  * 因此，我们只需遍历所有可能的单题轮 `p` (从 `1` 到 `m`)，计算答题范围包含 `p` 的选手数量，并取最大值。

结合以上两种情况，我们可以发现，样例提示中的划分（`{4}` 和 `{其他}`）就属于我们分析的情况二。

### 算法设计

综合上述分析，我们可以通过考察以下两种策略来找到最大晋级人数：

**策略 A (对应情况一):**
将题目分为 `[1, k]` 和 `[k+1, m]` 两轮。我们需要对所有可能的 `k` (从 `1`到 `m-1`) 计算晋级人数并找到最大值。
为了高效计算，我们可以：

1.  预处理所有 `L_i=1` 的选手，统计他们 `R_i` 的分布。通过后缀和，可以快速得到在 `L=1` 的前提下，`R>=k` 的选手数量。
2.  预处理所有 `R_i=m` 的选手，统计他们 `L_i` 的分布。通过前缀和，可以快速得到在 `R=m` 的前提下，`L<=k+1` 的选手数量。
3.  对于每个 `k`，晋级人数 = (满足 `L=1, R>=k` 的人数) + (满足 `L<=k+1, R=m` 的人数) - (满足 `L=1, R=m` 的人数，因为被重复计算)。
4.  整个过程的时间复杂度为 O(n+m)。

**策略 B (对应情况二):**
将题目分为 `{p}` 和 `其余所有题目` 两轮。我们需要对所有可能的 `p` (从 `1`到 `m`) 计算晋级人数并找到最大值。晋级人数就是答题范围 `[L, R]` 包含 `p` 的选手总数。
这个问题可以转化为：对于每个点 `p`，计算有多少个区间 `[L_i, R_i]` 覆盖了它。这是一个经典的区间覆盖问题，可以使用**差分数组**（或称扫掠线算法）高效解决：

1.  创建一个大小为 `m+2` 的差分数组 `diff`，初始化为0。
2.  遍历 `n` 个选手，对于每个区间 `[L, R]`，执行 `diff[L]++` 和 `diff[R+1]--`。
3.  计算 `diff` 数组的前缀和。前缀和数组中第 `p` 位的值，就等于覆盖点 `p` 的区间数量。
4.  遍历前缀和数组，找到最大值。
5.  整个过程的时间复杂度为 O(n+m)。

最终的答案就是从策略A和策略B中得到的最大晋级人数的较大者。

### C++ 解决方案

```cpp
#include <iostream>
#include <vector>
#include <algorithm>

using namespace std;

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(NULL);

    int n;
    long long m;
    cin >> n >> m;

    vector<pair<int, int>> contestants(n);
    vector<int> count_l1(m + 1, 0);
    vector<int> count_rm(m + 1, 0);
    int count_1m = 0;
    vector<long long> diff(m + 2, 0);

    for (int i = 0; i < n; ++i) {
        cin >> contestants[i].first >> contestants[i].second;
        int l = contestants[i].first;
        int r = contestants[i].second;

        // For Strategy B (Difference Array)
        diff[l]++;
        diff[r + 1]--;

        // For Strategy A (Preprocessing)
        if (l == 1) {
            count_l1[r]++;
        }
        if (r == m) {
            count_rm[l]++;
        }
        if (l == 1 && r == m) {
            count_1m++;
        }
    }

    int max_winners = 0;

    // --- Strategy B Calculation ---
    // Single problem {p} vs the rest
    long long current_coverage = 0;
    for (int p = 1; p <= m; ++p) {
        current_coverage += diff[p];
        max_winners = max(max_winners, (int)current_coverage);
    }

    // --- Strategy A Calculation ---
    // Partition [1, k] vs [k+1, m]
    vector<int> pass_r1_suffix_sum(m + 2, 0);
    for (int k = m; k >= 1; --k) {
        pass_r1_suffix_sum[k] = pass_r1_suffix_sum[k + 1] + count_l1[k];
    }

    vector<int> pass_r2_prefix_sum(m + 2, 0);
    for (int k = 1; k <= m; ++k) {
        pass_r2_prefix_sum[k] = pass_r2_prefix_sum[k - 1] + count_rm[k];
    }

    for (int k = 1; k < m; ++k) {
        int winners = pass_r1_suffix_sum[k] + pass_r2_prefix_sum[k + 1] - count_1m;
        max_winners = max(max_winners, winners);
    }
    
    // In case n=0, the result should be 0.
    if (n == 0) {
        max_winners = 0;
    }


    cout << max_winners << endl;

    return 0;
}
```

**代码说明:**

1.  我们同时为两种策略进行数据预处理。
2.  对于策略B，我们使用差分数组 `diff`。`diff[p]` 累加后的值 `current_coverage` 表示能答对题目 `p` 的人数。我们遍历所有 `p` 并更新最大值 `max_winners`。
3.  对于策略A，我们先计算出 `count_l1` 和 `count_rm` 数组。然后计算它们的前缀和与后缀和数组(`pass_r2_prefix_sum` 和 `pass_r1_suffix_sum`)。
4.  接着，我们遍历所有分割点 `k`，利用预处理好的数组 O(1) 地计算出当前 `k` 值的晋级人数，并更新 `max_winners`。
5.  最终输出 `max_winners`。

该算法的总时间复杂度为 O(n+m)，空间复杂度也为 O(n+m)，对于给定的数据范围是完全可行的。对于样例输入，程序将正确输出 `3`。

In [8]:
from typing import List

class Solution:
    def maxAdvance(self, n: int, m: int, intervals: List[List[int]]) -> int:
        # 处理 n=0 的边界情况
        if n == 0:
            print(0)
            return
            
        # --- 数据预处理 ---
        # 差分数组，用于策略B
        diff = [0] * (m + 2)
        # 统计L=1和R=m的选手，用于策略A
        count_l1 = [0] * (m + 1)
        count_rm = [0] * (m + 1)
        count_1m = 0 # 统计L=1且R=m的选手

        for l, r in intervals:
            # 为策略B填充差分数组
            diff[l] += 1
            if r + 1 <= m + 1:
                diff[r + 1] -= 1

            # 为策略A统计特定选手
            if l == 1:
                count_l1[r] += 1
            if r == m:
                count_rm[l] += 1
            if l == 1 and r == m:
                count_1m += 1

        max_winners = 0

        # --- 策略 B: 单题划分 {p} vs {其余} ---
        # 计算差分数组的前缀和，得到每个点p被覆盖的次数
        current_coverage = 0
        for p in range(1, m + 1):
            current_coverage += diff[p]
            # 此时 current_coverage 就是能答对题目 p 的人数
            # 这对应了 S2={p} 且 1 < p < m 的情况下的晋级人数
            # 我们在这里计算所有 p 的情况并取最大值，因为策略A会覆盖并修正边界情况
            max_winners = max(max_winners, int(current_coverage))


        # --- 策略 A: 连续区间划分 [1, k] vs [k+1, m] ---
        
        # 1. 计算后缀和: pass_r1[k] 表示 L=1 且 R>=k 的选手人数
        pass_r1_suffix_sum = [0] * (m + 2)
        for k in range(m, 0, -1):
            pass_r1_suffix_sum[k] = pass_r1_suffix_sum[k + 1] + count_l1[k]
        
        # 2. 计算前缀和: pass_r2[k] 表示 R=m 且 L<=k 的选手人数
        pass_r2_prefix_sum = [0] * (m + 2)
        for k in range(1, m + 1):
            pass_r2_prefix_sum[k] = pass_r2_prefix_sum[k - 1] + count_rm[k]

        # 3. 遍历所有分割点 k
        for k in range(1, m):
            # 晋级人数 = (通过第一轮) + (通过第二轮) - (两轮都通过，被重复计算)
            # 通过 [1, k] 的人数: L=1, R>=k -> pass_r1_suffix_sum[k]
            # 通过 [k+1, m] 的人数: L<=k+1, R=m -> pass_r2_prefix_sum[k+1]
            # 两轮都通过的人数: L=1, R=m -> count_1m
            winners = pass_r1_suffix_sum[k] + pass_r2_prefix_sum[k+1] - count_1m
            max_winners = max(max_winners, winners)

        return max_winners

In [9]:
n, m = 4, 8
intervals = [(4, 7), (1, 4), (5, 8), (2, 5)]
print(Solution().maxAdvance(n, m, intervals))  # 输出 3

3


为了解决这个问题，我们需要找到一种分配题目的方式，使得晋级的选手数量最大化。选手晋级的要求是至少有一轮的所有题目都在其答题范围内。经过分析，最优解通常可以通过将某一轮设置为单个题目（一个点）来实现，因为这样更容易被更多选手的答题范围覆盖。

### 方法思路
1. **问题分析**：选手的答题范围是连续的区间。为了最大化晋级人数，我们考虑将其中一轮设置为单个题目（一个点），另一轮为剩余的所有题目。选手晋级有两种情况：
   - 该选手的答题范围覆盖了单点题目（第一轮）。
   - 该选手的答题范围覆盖了剩余所有题目（第二轮），即该选手的答题范围必须是整个题目区间`[1, m]`。
   
   但是，覆盖整个题目区间的选手也会覆盖单点题目，因此在统计时不会重复计数。

2. **关键观察**：通过将某一轮设置为单个题目，我们可以统计覆盖每个题目的选手数量。最大覆盖数即为可能的最大晋级人数。这是因为：
   - 如果某选手覆盖了单点题目，他就能晋级。
   - 覆盖整个题目区间的选手在统计单点覆盖时已被计算。

3. **算法选择**：使用差分数组技术高效统计每个题目被多少选手覆盖。具体步骤：
   - 初始化一个长度为`m+2`的数组（考虑边界）。
   - 对于每个选手的区间`[L, R]`，执行`cover[L] += 1`和`cover[R+1] -= 1`。
   - 通过前缀和计算覆盖每个题目的选手数量。
   - 遍历所有题目，找到最大覆盖数。

4. **复杂度分析**：
   - **时间复杂度**：O(n + m)。其中，n 是选手数量，m 是题目数量。差分数组的构建和前缀和计算均为线性时间。
   - **空间复杂度**：O(m)。差分数组的空间开销。

### 解决代码
```python
import sys

def main():
    data = sys.stdin.read().split()
    n = int(data[0])
    m = int(data[1])
    cover = [0] * (m + 2)
    
    index = 2
    for _ in range(n):
        l = int(data[index])
        r = int(data[index + 1])
        index += 2
        cover[l] += 1
        if r + 1 <= m:
            cover[r + 1] -= 1
    
    for i in range(1, m + 1):
        cover[i] += cover[i - 1]
    
    ans = 0
    for i in range(1, m + 1):
        if cover[i] > ans:
            ans = cover[i]
    
    print(ans)

if __name__ == "__main__":
    main()
```

### 代码解释
1. **数据读取**：读取输入数据，包括选手数量`n`、题目数量`m`以及每个选手的答题范围`[L, R]`。
2. **差分数组初始化**：创建一个长度为`m+2`的数组`cover`，用于差分统计。
3. **差分更新**：对于每个选手的答题范围`[L, R]`：
   - 在`L`位置增加1（表示从`L`开始覆盖）。
   - 在`R+1`位置减少1（表示覆盖在`R+1`结束）。
4. **前缀和计算**：通过前缀和操作，将差分数组转换为每个题目被覆盖的选手数量。
5. **寻找最大值**：遍历所有题目，找到被最多选手覆盖的题目数量，即为最大晋级人数。
6. **输出结果**：打印最大晋级人数。

此方法高效地利用了差分数组技术，在线性时间内解决了问题，适用于大规模输入。